In [ ]:
import numpy as np
import faiss
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder
import openai

In [ ]:
# 1차 Recall - FAISS 기반 semantic search
def semantic_search(query, chunks, embedding_model, k=50):
    db = FAISS.from_documents(chunks, embedding_model)
    retriever = db.as_retriever(search_kwargs={"k": k})
    return retriever(query)

In [ ]:
# 키워드 필터 함수
def keyword_filter(docs, keywords):
    if not keywords:
        return docs
    filtered = []
    for doc in docs:
        if any(kw.lower() in doc.page_content.lower() for kw in keywords):
            filtered.append(doc)
    return filtered

In [ ]:
# Cross-Encoder 기반 Re-Ranking
def rerank(query, docs, top_k=10, model_name="cross-encoder/ms-marco-MiniLM-L-6-v2"):
    cross_encoder = CrossEncoder(model_name)
    pairs = [[query, doc.page_content] for doc in docs]
    scores = cross_encoder.predict(pairs)
    scored_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in scored_docs[:top_k]]

In [ ]:
# LLM 기반 3줄 요약(Map-Reduce)
def summarize_chunks(docs, question, openai_api_key, model="gpt-4.1-mini"):
    openai.api_key = openai_api_key
    summaries = []
    for doc in docs:
        prompt = f"다음 내용을 3줄로 요약해 주세요. 질문: {question}\n\n{doc.page_content}"
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=256,
            temperature=0.5,
        )
        summary = response['choices'][0]['message']['content']
        summaries.append(summary)
    # Reduce: 전체 요약
    final_prompt = f"아래 요약들을 종합해서 3줄로 다시 요약해 주세요.\n\n" + "\n\n".join(summaries)
    final_response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": final_prompt}],
        max_tokens=256,
        temperature=0.5,
    )
    return final_response['choices'][0]['message']['content']

In [ ]:
# 전체 파이프라인 함수
def rag_pipeline(query, chunks, embedding_model, openai_api_key, keywords=None):
    # 1차 Recall
    candidates = semantic_search(query, chunks, embedding_model, k=50)
    # 키워드 필터
    filtered = keyword_filter(candidates, keywords)
    # 2차 Re-Ranking
    reranked = rerank(query, filtered, top_k=10)
    # LLM Map-Reduce 요약
    summary = summarize_chunks(reranked, query, openai_api_key)
    return summary

In [ ]:
# 질문 입력 및 실행
question = input("질문을 입력하세요: ")
keywords = input("필터링할 키워드를 ,로 구분해서 입력하세요(없으면 엔터): ").split(",") if input("키워드 필터를 사용하시겠습니까? (y/n): ") == "y" else None
openai_api_key = "..."  # OpenAI API 키 입력

result = rag_pipeline(question, chunks, embedding_model, openai_api_key, keywords)
print("최종 3줄 요약 결과:\n", result)